In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

* $h:$ coordenada vertical del fondo marino, $>0$ en el agua $<0$ en tierra
* $\eta:$ coordenada vertical de la superficie libre, $>0$ implica el agua está sobre el nivel medio del mar, $\eta=-h$ en tierra.
* $H=h+\eta$ altura de la columna del agua, $H=0$ en tierra
* el $0$ vertical está en el nivel medio del agua WML

In [ ]:
nt = 100
nx = 100
x = np.linspace(0,100,nx)
eta = np.zeros((nt,nx))
zdepth = np.zeros_like(eta)
p = np.zeros((nt,nx))
h = np.zeros_like(x)

h = h +1.0
#eta[0,:] = 0.05*np.exp(-(x-50)**2/50) ##gaussian bell
#eta[0,:] = np.where(np.abs(x-50)<20, 0.04, 0.0) #square
eta[0,:] = 1/8.*np.where(np.abs(x-50)<20,np.exp(-1./(1-((np.abs(x)-50)/20)**2)),0.0) #mollifier bell
zdepth[0,:] = eta[0,:]+h

plt.plot(x,eta[0,:])
# plt.plot(x,-h)
plt.ylim(-0.05,0.05)

In [ ]:
cfl = 0.9
dx = np.diff(x)[0]
dt = cfl*dx/np.sqrt(np.max(h)*9.81)
rx = dt/dx
gx = 1e-5
grx = 9.81*dt/dx

## Simulacion: sin batimetria

In [ ]:
dp1 = np.zeros_like(x)
dp2 = np.zeros_like(x)
for n in range(nt-1):
    ## Mas Loop
    for i in range(1, nx-1):
        zzz = eta[n,i] - rx*(p[n,i]-p[n,i-1])
        DD = zzz + h[i]
        zdepth[n+1,i] = DD
        eta[n+1,i] = zzz    


    ## Calculate total water depth and total water depth a discharge point
    for i in range(nx):
        ip1 = min(nx-1,i+1)
        dp1[i] = 0.25*(zdepth[n+1,i] + zdepth[n,i] + zdepth[n+1,ip1] + zdepth[n, ip1])
        dp2[i] = 0.5*(zdepth[n+1,i]+zdepth[n+1,ip1])    

    # open boundary
    i = 0
    if h[i]>gx:
        cc = np.sqrt(9.81*h[i])
        uh = 0.0 #de acuerdo al codigo fortran
        uu = np.sqrt(uh**2+p[n,i]**2)
        zz = uu/cc
        if p[n,i]>0:
            zz = -zz
        eta[n+1,i] = zz
        zdepth[n+1,i] = zz+h[i]#esto no esta en el codigo original

    i = nx-1
    if h[i]>gx:
        cc = np.sqrt(9.81*h[i])
        uh = 0.0 #de acuerdo al codigo fortran
        uu = np.sqrt(uh**2+p[n,i-1]**2)
        zz = uu/cc
        if p[n,i-1]<0:
            zz = -zz
        eta[n+1,i] = zz     
        zdepth[n+1,i] = zz+h[i]##esto no esta en el codigo original
        
    # momentum loop
    for i in range(0,nx-1):
        dd = dp2[i]
        df = dp1[i]
        p[n+1,i] = p[n,i] - 9.81*rx*dd*(eta[n+1,i+1]-eta[n+1,i])



        
    #sponge layer
#     for i in range(nx):
#         p[n+1,i] = p[n+1,i] - sponge_coefx[i]*dt*p[n,i]
    

# JSAnimation

In [ ]:
from JSAnimation import IPython_display
from matplotlib import animation

# create a simple animation
fig = plt.figure(figsize=(12,4))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
line1, = ax1.plot([], [], lw=2)
line2, = ax2.plot([], [], lw=2)

ax1.set_xlim(0,100)
ax1.set_ylim(0.99-1,1.1-1)
ax2.set_xlim(0,100)
ax2.set_ylim(-1,1)
def init():
    line1.set_data([], [])
    line2.set_data([],[])
    return line1,line2

def animate(i):
    line1.set_data(x, eta[i*1,:])
    line2.set_data(x, p[i*1,:])
    return line2,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=100, interval=20, blit=True)

El borde izquierdo no funciona realmente

In [ ]:
plt.plot(p[:,0])

## Sponge layer coef

In [ ]:
def sponge_coef(x,H):
    sponge_coefx = np.zeros_like(x)
    wavelength = 1.
    width = 20*wavelength
    alpha_c = 10.0
    alpha_mu = 0.0
    M = 2
    T = wavelength/np.sqrt(9.81*H)
    omega = 2.0*np.pi/T

    sponge_width = width
    rmax = width

    xs = x[0]+sponge_width
    xe = x[-1]-sponge_width

    for i in range(nx):
        if (x[i]<xs):
            rx = xs-x[i]
        elif x[i]>=xe:
            rx = x[i]-xe
        else:
            rx = 0.0
        r = rx
        xrel = rx 
        if xrel<=0:
            xrel = 0.0
        elif xrel >= rmax:
            xrel  = rmax

        sponge_coefx[i] = alpha_c*omega*(np.exp((xrel/rmax)**2)-1)/(np.exp(1)-1.0)
    return sponge_coefx

In [ ]:
sponge_coefx = sponge_coef(x,h[0])
plt.plot(x,sponge_coefx)